## 학원 규모가 커지면서 시스템을 확장하려 한다. 확장 전 학원의 등록된 학생들의 주요 정보를 파악하려 한다.

enrollment_info.csv

|컬럼|정의|type|
|:---|:---|:---:|
|enrollment_code|학생 등록 코드(길이6 알파벳 대문자)|object|
|gender|여자면 F, 남자면 M|object|
|bonus|상점|float64|
|penalty|벌점|float64|
|school_code|소속 학교 코드|object|  

정답 및 해설 : https://tjd229.tistory.com/20

In [22]:
import numpy as np
import pandas as pd

In [23]:
df = pd.read_csv("../Dataset/enrollment_info.csv")
print(df.dtypes)
print(df.shape)
df.head()

enrollment_code     object
gender              object
bonus              float64
penalty            float64
school_code         object
dtype: object
(86550, 5)


,enrollment_code,gender,bonus,penalty,school_code
0,QEHRNZ,F,0.0,0.0,SWHS
1,IUGTFR,F,1.6,0.0,PYHS
2,PWJCQA,M,0.0,0.0,NKHS
3,ZYFHCX,F,5.5,0.0,NKHS
4,KIBAHX,M,0.6,0.0,MWHS


### Q1. 다음 단계에 따라 분석을 수행하고 질문에 답하시오.
단계 1 : 각 학생들의 상점(bonus)에서 벌점(penalty)을 뺀 값을 합계 점수로 정의한다.  
단계 2 : 각 학교 코드(school_code)별 합계 점수 평균을 구하고, 이 중 3번째로 큰 값을 A라고 한다.  
단계 3 : 0미만의 합계 점수는 0으로 변경 후, 각 학교 코드(school_code)별 합계 점수 평균을 구하고, 이 중 3번째로 큰 값을 B라고 한다.  

단계 2에서 구한 A와 단계 3에서 구한 B의 절대값 차이를 구하시오  
※ 결과는 반올림하여 소수점 첫째 자리까지 기술하시오. (답안예시 : 22.9)


In [24]:
df1 = df.copy()
df1['tot'] = df1['bonus'] - df1['penalty']
df1

gb1 = df1.groupby('school_code')['tot'].mean().sort_values(ascending = False)

A = gb1.iloc[3]
A

df1['tot'] = df1['tot'].apply(lambda x: 0 if x < 0 else x)
df1
gb2 = df1.groupby('school_code')['tot'].mean().sort_values(ascending = False)

B = gb2.iloc[3]
B

round(abs(A-B), 1)


0.1

### Q2. 데이터에는 총 6종류의 학교 코드(school_code)가 있다. 학생들의 등록 코드(enrollment_code)에 학교 코드(school_code)가 포함되어 있는 경우, 시스템에 문제가 발생할 수 있어, 이를 사전에 방지하고자 한다. 등록 코드(enrollment_code)에 학교 코드(school_code)가 포함되어 있는 학생들의 수 총합을 구하시오


단계 1 : 6종류의 학교 코드(school_code)를 구한다.  
단계 2 : 등록 코드(enrollment_code)에 단계 1에서 구한 6종류의 학교 코드 중 하나가 포함되어 있는 학생 수를 구하시오  

※ 학교 코드(school_code)가 **ZXCV**일 때, 등록 코드(enrollment_code)가 **ZXCV**AB, A**ZXCV**B, CD**ZXCV** 등의 경우는 등록 코드(enrollment_code)에 학교 코드(school_code)가 포함되어 있다고 본다.  
※ 학교 코드(school_code)가 **ZXCV**일 때, 등록 코드(enrollment_code)가 **ZX**AB**CV**의 경우는 등록 코드(enrollment_code)에 학교 코드(school_code)가 포함되어 있다고 보지 않는다.  



In [25]:
df2 = df.copy()

school_list = list(df2['school_code'].unique())
school_list

n_student_list = []

# ans = 0;
# for sch in school_list:
#   mask = df2['enrollment_code'].str.contains(sch)
#   ans += sum(mask)
# print(ans)

ans = 0;
masklist = []
for sch in school_list:
  mask = df2['enrollment_code'].str.contains(sch)
  # print(type(mask))
  ans += sum(mask)
  masklist.append(mask)

print(ans)
  

3


### Q3. 학생들의 학업성취도를 분석하기 위해 군집화를 하려고 한다. 다음 단계에 따라 분석을 수행하고 질문에 답하시오.

단계 1 : 등록 코드(enrollment_code)에 'HS'가 포함되어 있지 않은 학생들을 제거한다.  
단계 2 : 성별(gender) 컬럼에서 "M"을 0, "F"를 1로 변환한다.  
단계 3 : 독립 변수들에 대해 K-means 군집 분석을 수행한다. 이 때, 군집 수는 2~9개 중 K-means Silhouette 를 통해 구하고, 이 중 두 번째로 높은 score를 최적의 K로 설정한다.  
- 독립 변수(총 3개) : 성별(gender), 상점(bonus), 벌점(penalty)  

단계 4 : 단계 3에서 최적의 K로 도출한 각 군집에 소속된 학생 수의 비율 중, 가장 큰 값을 구하시오  

※ 결과는 반올림하여 소수점 둘째 자리까지 기술하시오. (답안예시 : 2.29)



In [39]:
from sklearn.cluster import KMeans
from sklearn.metrics import silhouette_score

df3= df.copy()

df3_1 = df3[df3['enrollment_code'].str.contains('HS')]


df3_1['gender'] = df3_1['gender'].apply(lambda x: 1 if x == 'F' else 0)
df3_1
# df3_2 = df3_1.loc[:,'gender':'penalty']
df3_2 = df3_1.loc[:,['gender','bonus','penalty']]


scores = []
klist = []
for k in range(2,10):
  model = KMeans(n_clusters=k, random_state=229).fit(df3_2)
  score = silhouette_score(df3_2, model.labels_)
  scores.append(score)
  klist.append(k)

data = {'cluster':klist, 'sco': scores}
df3_3 = pd.DataFrame(data)
# df3_3.sort_values('sco', ascending=False)
df3_3
#print(scores)  
  


/tmp/ipykernel_14261/4050022917.py:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df3_1['gender'] = df3_1['gender'].apply(lambda x: 1 if x == 'F' else 0)
/home/codespace/.local/lib/python3.10/site-packages/sklearn/cluster/_kmeans.py:1416: FutureWarning: The default value of `n_init` will change from 10 to 'auto' in 1.4. Set the value of `n_init` explicitly to suppress the warning
  super()._check_params_vs_input(X, default_n_init=10)
/home/codespace/.local/lib/python3.10/site-packages/sklearn/cluster/_kmeans.py:1416: FutureWarning: The default value of `n_init` will change from 10 to 'auto' in 1.4. Set the value of `n_init` explicitly to suppress the warning
  super()._check_params_vs_input(X, default_n_init=10)
/home/codespace/.local/lib/pytho

,cluster,sco
0,2,0.493727
1,3,0.404853
2,4,0.367341
3,5,0.378767
4,6,0.377287
5,7,0.395732
6,8,0.418087
7,9,0.436181


#### 필요 라이브러리 함수,클래스 및 설정값 목록  

from sklearn.cluster import KMeans  
from sklearn.metrics import silhouette_score  
random_state = 229  
문제 지시 외 Default 값 사용  